In [ ]:
"""

Darstellung der BoXPol RHI Scans bei einem GPM Overpass

"""
print ('_')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import wradlib
import h5py
import glob

In [ ]:
#ZP = '20141007'
ZP = '20140826'

print 'Zeitpunkt: ',ZP

year, m, d, ht, mt, st = ZP[0:4], ZP[4:6], ZP[6:8], ZP[8:10], ZP[10:12], ZP[12:14]
ye = ZP[2:4]

nr = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15']

for jj in range(len(nr)):
    if year < '2015':
        print 'archive'
        sc = 'radar-archiv'
    else:
        sc = 'radar'
    ################################################### Read RADOLAN GK Koordinaten

    iii = 0
    pfad = glob.glob('/automount/'+sc+'/scans/'+str(year)+'/'+str(year)+'-'+str(m)+'/'+
            str(year)+'-'+str(m)+'-'+str(d)+'/gpm_rhi_'+nr[jj]+'/'+'*.mvol')[0]
    print pfad

    #Todo: die an der Uhrzeit am nachliegensten Uhrzeit nehmen

    boxpol_filename = wradlib.util.get_wradlib_data_file(pfad)



    ppi=h5py.File(boxpol_filename,'r')
    data, attrs = wradlib.io.read_GAMIC_hdf5(boxpol_filename)

    print data[u'SCAN0'].keys()
    print attrs['VOL'].keys()
    print attrs['SCAN0'].keys()


    zh = data['SCAN0'][u'ZH']['data']
    phidp = data['SCAN0'][u'PHIDP']['data']
    rhohv = data['SCAN0'][u'RHOHV']['data']
    zv = data['SCAN0'][u'ZV']['data']
    zdr = data['SCAN0'][u'ZDR']['data']
    kdp = data['SCAN0'][u'KDP']['data']

    r = attrs['SCAN0']['r']
    az = attrs['SCAN0']['az']
    print az
    th = attrs['SCAN0']['el']

    lon_ppi = attrs['VOL']['Longitude']
    lat_ppi = attrs['VOL']['Latitude']
    alt_ppi = attrs['VOL']['Height']

    from mpl_toolkits.axisartist.grid_finder import FixedLocator, DictFormatter
    # reading in GAMIC hdf5 file

    mask_ind = np.where(data <= np.nanmin(zh))
    zh[mask_ind] = np.nan
    ma = np.ma.array(zh, mask=np.isnan(zh))


    fft = 15
    import pcc as pcc
    cmap2 = pcc.get_miub_cmap()

    sb = [231,232,233,234,235,236]
    radpar = [zh, zv, zdr,phidp, rhohv, kdp]
    vmini = [0, 0, -1, -100, .7,-0.75]
    vmaxi = [50, 50, 3, 50, .99, 2]
    name = ['Zh (dBZ)','Zv (dBZ)','Zdr (db)','PhiDP','RhoHV (/)', r"kdp ( $^\circ$ $km^{-1}$)"]
    #cc = [cmap2, cmap2, cmap2, 'jet', 'jet', 'jet']
    fig = plt.figure(figsize=(20,8))

    for iii in range(len(radpar)):

        cgax, caax, paax, pm, xxx, yyy = pcc.pcc_plot_cg_rhi(radpar[iii], r=r,
                                                             th=th,
                                                             rf=1e3,
                                                             subplot=sb[iii],
                                                             autoext=True,
                                                             vmin=vmini[iii],
                                                             vmax= vmaxi[iii],
                                                             cmap=cmap2)
        cgax.set_ylim(0,7)
        cbar = plt.gcf().colorbar(pm, shrink=0.8, extend='both')#, pad=0.05)
        plt.gca().invert_xaxis()
        cbar.set_label(name[iii],fontsize=fft)
        caax.set_xlabel('x (km)',fontsize=fft)
        caax.set_ylabel('z (km)',fontsize=fft)

        cbar.ax.tick_params(labelsize=fft)

        caax.tick_params(labelsize=fft)

        plt.title(' BoXPol RHI: '+ ZP, fontsize=fft)


        gh = cgax.get_grid_helper()
        locs = [0.]
        gh.grid_finder.grid_locator1 = FixedLocator(locs)
        gh.grid_finder.tick_formatter1 = DictFormatter(dict([(i, r"${0:.0f}^\circ$".format(i)) for i in locs]))
        plt.yticks(fontsize=fft)
        plt.xticks(fontsize=fft)


    texti = 'Azimuth :' + str(attrs['SCAN0']['azimuth'])
    plt.figtext(0, 0, texti)
    plt.tight_layout()
    #plt.show()
    plt.savefig('/automount/ags/velibor/plot/GPM_boxpol/topevents/boxpol_gpm_rhi_'+ZP+'_'+nr[jj]+'.png')
    plt.close()

    print attrs['SCAN0']['azimuth']